In [1]:
from io import open
import unicodedata
import string
import pickle
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

SOS_token = 0
EOS_token = 1

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
SOS_token = 0
EOS_token = 1
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
    )

Модифицируем функцию чтения данных

In [5]:
def normalizeString(s, rus = False):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"([\s+])", r" ", s)
    if rus:
        s = re.sub(r"[^а-яА-ЯёЁ.!?]+", r" ", s)
    else:
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    if lang2 is not None:
        lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
        pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    else:
        lines = open('data/%s.txt' % (lang1), encoding='utf-8').read().strip().split('\n')
        lines = [[s for i, s in enumerate(l.split('\t')) if i in [0,1] ] for l in lines]
        pairs = [[ normalizeString(l[0]),normalizeString(l[1], rus=True) ] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs


In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2 =None, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)

    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [8]:
input_lang, output_lang, pairs = prepareData('rus', None, True)

Reading lines...
Read 496059 sentence pairs
Trimmed to 28719 sentence pairs
Counting words...
Counted words:
None 10174
rus 4303


In [9]:
pairs[-5:]

[['она улыбнулась в ответ на его нежныи взгляд .',
  'she smiled in response to his affectionate glance .'],
 ['она проводит каждое воскресенье со своеи бабушкои .',
  'she spends time with her grandmother every sunday .'],
 ['каждую субботу после обеда она играет в теннис .',
  'she spends every saturday afternoon playing tennis .'],
 ['после аварии она перестала бывать на людях .',
  'she stopped appearing in public after her accident .'],
 ['они ведут переговоры чтобы приити к приемлемому компромиссу .',
  'they are negotiating to reach a satisfactory compromise .']]

## Модель

In [22]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder( decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [14]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [15]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [16]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)



In [33]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 41s (- 9m 45s) (5000 6%) 3.1193
1m 19s (- 8m 36s) (10000 13%) 2.6248
1m 57s (- 7m 48s) (15000 20%) 2.3431
2m 34s (- 7m 5s) (20000 26%) 2.1771
3m 12s (- 6m 25s) (25000 33%) 1.9861
3m 50s (- 5m 46s) (30000 40%) 1.8850
4m 28s (- 5m 7s) (35000 46%) 1.7698
5m 6s (- 4m 28s) (40000 53%) 1.6245
5m 44s (- 3m 49s) (45000 60%) 1.5321
6m 22s (- 3m 11s) (50000 66%) 1.4711
7m 0s (- 2m 33s) (55000 73%) 1.4310
7m 38s (- 1m 54s) (60000 80%) 1.3418
8m 16s (- 1m 16s) (65000 86%) 1.2850
8m 54s (- 0m 38s) (70000 93%) 1.2439
9m 32s (- 0m 0s) (75000 100%) 1.2127


In [37]:
evaluateRandomly(encoder1, decoder1)

> вы станете знаменитым .
= you re going to be famous .
< you re going to be famous . <EOS>

> мы помолвлены .
= we re engaged .
< we re going . <EOS>

> ты ослеплен любовью .
= you are blinded by love .
< you re my a . <EOS>

> я всегда озадачен .
= i m always confused .
< i m always tired . <EOS>

> я очень больнои человек .
= i m a very sick person .
< i m a very busy person . <EOS>

> я не такая высокая как ты .
= i m not as tall as you .
< i m not as tall as you . <EOS>

> они очень важны .
= they are very important .
< they are very important . <EOS>

> я занята так что не могу поити .
= i m busy so i can t go .
< i m busy so i can t go . <EOS>

> я собираюсь в бостон этои осенью .
= i m going to boston this fall .
< i m going to go this this . <EOS>

> я рад что мы можем помочь .
= i m glad we can help .
< i m glad we can help . <EOS>



## Попробуем добавить +1 рекуррентный слой в encoder и decoder

In [40]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size , num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [41]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 51s (- 12m 7s) (5000 6%) 3.1276
1m 40s (- 10m 53s) (10000 13%) 2.7179
2m 28s (- 9m 53s) (15000 20%) 2.4281
3m 16s (- 8m 59s) (20000 26%) 2.2424
4m 4s (- 8m 8s) (25000 33%) 2.0526
4m 51s (- 7m 17s) (30000 40%) 1.9292
5m 40s (- 6m 29s) (35000 46%) 1.8049
6m 28s (- 5m 39s) (40000 53%) 1.7041
7m 18s (- 4m 52s) (45000 60%) 1.6100
8m 6s (- 4m 3s) (50000 66%) 1.5015
8m 55s (- 3m 14s) (55000 73%) 1.4615
9m 43s (- 2m 25s) (60000 80%) 1.3626
10m 32s (- 1m 37s) (65000 86%) 1.3109
11m 21s (- 0m 48s) (70000 93%) 1.2581
12m 9s (- 0m 0s) (75000 100%) 1.1912


In [42]:
evaluateRandomly(encoder1, decoder1)

> сожалею о прошлои ночи .
= i m sorry about last night .
< i m sorry about about this . . <EOS>

> я не ревнивыи человек .
= i m not a jealous person .
< i m not a person . <EOS>

> ты у меня в руках .
= you re in my hands .
< you re in my room . <EOS>

> он здесь чтобы следить за нами .
= he s here to spy on us .
< he is here to to our job . <EOS>

> они холодные .
= they re cold .
< they re getting . <EOS>

> мы самостоятельная нация .
= we are an independent nation .
< we re in a a . . <EOS>

> я кое кого жду .
= i m expecting someone .
< i m expecting for . . <EOS>

> они мне не враги .
= they re not my enemies .
< they re not my . . <EOS>

> я ничего не выбрасываю .
= i m not throwing anything away .
< i m not doing anything . <EOS>

> это все я сделал .
= i m the one who did all this .
< i m the one who did that . <EOS>



### Добавим вместо слоя GRU слой LSTM

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, -1)
        output = embedded

        output, (hn, cn) = self.lstm(output, (hidden[0], hidden[1]))
        return output, (hn, cn)

    def initHidden(self):
        return (
            torch.zeros( 1, self.hidden_size, device=device),
            torch.zeros( 1, self.hidden_size, device=device),
        )

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.LSTM(hidden_size, hidden_size )
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, -1)
        output = F.relu(output)
        output, (hidden_, c_n)  = self.gru(output, (hidden[0], hidden[1]))
        output = self.softmax(self.out(output))
        return output, (hidden_, c_n)

In [19]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 44s (- 10m 22s) (5000 6%) 3.2158
1m 25s (- 9m 14s) (10000 13%) 2.7478
2m 6s (- 8m 27s) (15000 20%) 2.5676
2m 47s (- 7m 39s) (20000 26%) 2.3289
3m 28s (- 6m 57s) (25000 33%) 2.2117
4m 10s (- 6m 15s) (30000 40%) 2.0843
4m 51s (- 5m 33s) (35000 46%) 1.9667
5m 32s (- 4m 51s) (40000 53%) 1.9064
6m 13s (- 4m 9s) (45000 60%) 1.8037
6m 55s (- 3m 27s) (50000 66%) 1.7326
7m 37s (- 2m 46s) (55000 73%) 1.6606
8m 18s (- 2m 4s) (60000 80%) 1.6214
8m 59s (- 1m 22s) (65000 86%) 1.5491
9m 40s (- 0m 41s) (70000 93%) 1.4801
10m 22s (- 0m 0s) (75000 100%) 1.4216


In [20]:
evaluateRandomly(encoder1, decoder1)

> я не большои любитель баскетбола .
= i m not a big fan of basketball .
< i m not a big of of . . <EOS>

> они удивительно хороши .
= they re surprisingly good .
< they re waiting they . <EOS>

> ты не такои маленькии как я .
= you aren t as short as i am .
< you aren t like me as i am . <EOS>

> я такое трепло .
= i m such a blabbermouth .
< i m the the one who understands . <EOS>

> она член этои организации .
= she is a member of this organization .
< she is a member of this . . <EOS>

> ты снова врешь мне .
= you re lying to me again .
< you re lying to me again again . <EOS>

> я вряд ли выиграю .
= i m not likely to win .
< i m not likely to be . <EOS>

> он носитель англииского языка .
= he is a native english speaker .
< he is a new and . . <EOS>

> вы дебилы .
= you re morons .
< you re shy . <EOS>

> им всегда не хватает денег .
= they are always short of money .
< they re always trying money . <EOS>



# Вывод
- обучили модель перевода с руского на английский
- модель на основе одного слоя GRU , двух слоев GRU, одного слоя LSTM
Качество не очень хорошее (по смыслу предложения близки у всех, но минимальная ошибка у модели с двумя GRU слоями)